In [ ]:
!pip install transformers==4.35.0
!pip install quanto==0.0.11
!pip install torch==2.1.1

## T5-FLAN

Due to hardware memory constraints the T5-FLAN model is shortlisted


In [ ]:
!pip install sentencepiece==0.2.0

### Without Quantization

In [ ]:
model_name = "google/flan-t5-small"

In [ ]:
import sentencepiece as spm
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
input_text = "Hello, my name is "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
from helper import compute_module_sizes

module_sizes = compute_module_sizes(model)
print(f"The model size is {module_sizes[''] * 1e-9} GB")

### Quantize the model (8-bit precision)

In [ ]:
from quanto import quantize, freeze
import torch

In [ ]:
quantize(model, weights=torch.int8, activations=None)

In [ ]:
print(model)

### Freeze the model

In [ ]:
freeze(model)

In [ ]:
module_sizes = compute_module_sizes(model)
print(f"The model size is {module_sizes[''] * 1e-9} GB")

In [ ]:
input_text = "Hello, my name is "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

## EleutherAI/pythia-410m

Load [EleutherAI/pythia-410m](https://huggingface.co/EleutherAI/pythia-410m) model and tokenizer.

### Without Quantization



In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

model_name = "EleutherAI/pythia-410m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True)
print(model.gpt_neox)

In [ ]:
text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=10)
outputs
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
from helper import compute_module_sizes

# compute model size
module_sizes = compute_module_sizes(model)
print(f"The model size is {module_sizes[''] * 1e-9} GB")
print(model.gpt_neox.layers[0].attention.dense.weight)

**Note:** The weights are in `fp32`

### 8-bit Quantization

Requires 8GB RAM

In [ ]:
from quanto import quantize, freeze
import torch

quantize(model, weights=torch.int8, activations=None)
# after performing quantization
print(model.gpt_neox)
print(model.gpt_neox.layers[0].attention.dense.weight)

In [ ]:
# freeze the model
freeze(model)

In [ ]:
print(model.gpt_neox.layers[0].attention.dense.weight)

# get model size after quantization
module_sizes = compute_module_sizes(model)
print(f"The model size is {module_sizes[''] * 1e-9} GB")

In [ ]:
# run inference after quantizing the model
outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#### Comparing "linear quantization" to "downcasting"

- When downcasting a model, you convert the model's parameters to a more compact data type (bfloat16).  During inference, the model performs its calculations in this data type, and its activations are in this data type.  Downcasting may work with the bfloat16 data type, but the model performance will likely degrade with any smaller data type, and won't work if you convert to an integer data type (like the int8 in this lesson).


- Linear quantization enables the quantized model to maintain performance much closer to the original model by converting from the compressed data type back to the original FP32 data type during inference. So when the model makes a prediction, it is performing the matrix multiplications in FP32, and the activations are in FP32.  This enables you to quantize the model in data types smaller than bfloat16, such as int8